In [138]:
import pandas as pd
from rdkit import Chem
from fragmenstein import Laboratory, Wictor
from fragmenstein.laboratory.validator import place_input_validator
import os, logging, time
from typing import List
import ast

In [139]:
# Load the data
df = pd.read_pickle('bases_to_rerun.pkl')
df['name'] = df['base_output_dir']
df.drop(columns=['base_output_dir'], inplace=True)
df

,batch,base,minimised,error,smiles,hits,name
0,batch1,IGUPJTYXYZPOEC-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0310A+A71EV2A-x0732A,N#CCC(=O)Nc1cccc(NC(=O)CO)c1,x0310A x0732A,IGUPJTYXYZPOEC-UHFFFAOYSA-N-4Sratv-base-A
1,batch1,NBJGZNPZLLTJSQ-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0310A+A71EV2A-x0732A,Cc1ccc(NC(=O)CC#N)cc1NC(=O)CO,x0310A x0732A,NBJGZNPZLLTJSQ-UHFFFAOYSA-N-4DiKxh-base-A
2,batch1,ODONBPGOOMWOPQ-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0310A+A71EV2A-x0732A,N#CCC(=O)Nc1ccnc(NC(=O)CO)c1,x0310A x0732A,ODONBPGOOMWOPQ-UHFFFAOYSA-N-nEzAW3-base-A
3,batch1,TYJSTJNCXZDPLY-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0310A+A71EV2A-x0732A,Cc1ccc(NC(=O)CO)cc1NC(=O)CC#N,x0310A x0732A,TYJSTJNCXZDPLY-UHFFFAOYSA-N-CMcr7A-base-A
5,batch1,JVLRHPWRWSSGDH-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0310A+A71EV2A-x0732A,N#CCC(=O)Nc1cc(NC(=O)CO)ccc1F,x0310A x0732A,JVLRHPWRWSSGDH-UHFFFAOYSA-N-SYTESp-base-A
6,batch1,MBMRUNGQBMGNOP-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0310A+A71EV2A-x0732A,Cc1cc(NC(=O)CO)cc(NC(=O)CC#N)c1,x0310A x0732A,MBMRUNGQBMGNOP-UHFFFAOYSA-N-3ncoui-base-A
8,batch3,QDEXMOJNLWMDJW-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0528A,CN1CCC(Oc2ccccc2OCC(=O)N2CCOCC2)C1=O,x0528A x0739A,QDEXMOJNLWMDJW-UHFFFAOYSA-N-3nJyVp-base-B
9,batch3,QDEXMOJNLWMDJW-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0528A,CN1CCC(Oc2ccccc2OCC(=O)N2CCOCC2)C1=O,x0528A x0739A,QDEXMOJNLWMDJW-UHFFFAOYSA-N-3nJyVp-base-A
10,batch3,BRPRTCVLXGUWHL-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0528A,Cc1ccc(OC2CCN(C)C2=O)c(OCC(=O)N2CCOCC2)c1,x0528A x0739A,BRPRTCVLXGUWHL-UHFFFAOYSA-N-3QVTyu-base-B
11,batch3,BRPRTCVLXGUWHL-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0528A,Cc1ccc(OC2CCN(C)C2=O)c(OCC(=O)N2CCOCC2)c1,x0528A x0739A,BRPRTCVLXGUWHL-UHFFFAOYSA-N-3QVTyu-base-A


In [140]:
Wictor.monster_throw_on_discard = True  # stop this merger if a fragment cannot be used.
Wictor.monster_joining_cutoff = 5  # Å
Wictor.quick_reanimation = False  # for the impatient
Wictor.error_to_catch = Exception  # stop the whole laboratory otherwise
Wictor.enable_stdout(logging.CRITICAL)
Laboratory.Victor = Wictor

In [141]:
# add hits_path
def add_hits(input_df: pd.DataFrame) -> pd.DataFrame:
    """
    This function adds the hits_path as mol objects to input_df['hits_path'].
    """
    df['hits_path'] = None
    hits_path = '/Users/kate_fieseler/PycharmProjects/EV-A71-2A-syndirella-run/fragments/new_hits.sdf'
    # load hits_path either from mol or sdf
    if os.path.splitext(hits_path)[1] == '.mol':
        print('This is a mol file')
        hits: List[Chem.Mol] = [Chem.MolFromMolFile(hits_path.strip())]
    else:
        with Chem.SDMolSupplier(hits_path.strip()) as sd:
            hits: List[Chem.Mol] = list(sd)
    for i, row in input_df.iterrows():
        hits_names = row['hits'].split(' ')
        print(hits_names)
        # Filter hits based on hits_names
        filtered_hits = [
        hit for hit in hits
        if any(hit_name in hit.GetProp('_Name').split('-')[1] for hit_name in hits_names)
        ]
        print(filtered_hits)
        input_df.at[i, 'hits_path'] = filtered_hits
    df['hits'] = df['hits_path']
    df.drop(columns=['hits_path'], inplace=True)
    return input_df
add_hits(df)
df

['x0310A', 'x0732A']
[<rdkit.Chem.rdchem.Mol object at 0x29a203f90>, <rdkit.Chem.rdchem.Mol object at 0x299582f20>]
['x0310A', 'x0732A']
[<rdkit.Chem.rdchem.Mol object at 0x29a203f90>, <rdkit.Chem.rdchem.Mol object at 0x299582f20>]
['x0310A', 'x0732A']
[<rdkit.Chem.rdchem.Mol object at 0x29a203f90>, <rdkit.Chem.rdchem.Mol object at 0x299582f20>]
['x0310A', 'x0732A']
[<rdkit.Chem.rdchem.Mol object at 0x29a203f90>, <rdkit.Chem.rdchem.Mol object at 0x299582f20>]
['x0310A', 'x0732A']
[<rdkit.Chem.rdchem.Mol object at 0x29a203f90>, <rdkit.Chem.rdchem.Mol object at 0x299582f20>]
['x0310A', 'x0732A']
[<rdkit.Chem.rdchem.Mol object at 0x29a203f90>, <rdkit.Chem.rdchem.Mol object at 0x299582f20>]
['x0528A', 'x0739A']
[<rdkit.Chem.rdchem.Mol object at 0x299582660>, <rdkit.Chem.rdchem.Mol object at 0x299582c80>]
['x0528A', 'x0739A']
[<rdkit.Chem.rdchem.Mol object at 0x299582660>, <rdkit.Chem.rdchem.Mol object at 0x299582c80>]
['x0528A', 'x0739A']
[<rdkit.Chem.rdchem.Mol object at 0x299582660>, <rd

,batch,base,minimised,error,smiles,hits,name
0,batch1,IGUPJTYXYZPOEC-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0310A+A71EV2A-x0732A,N#CCC(=O)Nc1cccc(NC(=O)CO)c1,[<rdkit.Chem.rdchem.Mol object at 0x29a203f90>...,IGUPJTYXYZPOEC-UHFFFAOYSA-N-4Sratv-base-A
1,batch1,NBJGZNPZLLTJSQ-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0310A+A71EV2A-x0732A,Cc1ccc(NC(=O)CC#N)cc1NC(=O)CO,[<rdkit.Chem.rdchem.Mol object at 0x29a203f90>...,NBJGZNPZLLTJSQ-UHFFFAOYSA-N-4DiKxh-base-A
2,batch1,ODONBPGOOMWOPQ-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0310A+A71EV2A-x0732A,N#CCC(=O)Nc1ccnc(NC(=O)CO)c1,[<rdkit.Chem.rdchem.Mol object at 0x29a203f90>...,ODONBPGOOMWOPQ-UHFFFAOYSA-N-nEzAW3-base-A
3,batch1,TYJSTJNCXZDPLY-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0310A+A71EV2A-x0732A,Cc1ccc(NC(=O)CO)cc1NC(=O)CC#N,[<rdkit.Chem.rdchem.Mol object at 0x29a203f90>...,TYJSTJNCXZDPLY-UHFFFAOYSA-N-CMcr7A-base-A
5,batch1,JVLRHPWRWSSGDH-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0310A+A71EV2A-x0732A,N#CCC(=O)Nc1cc(NC(=O)CO)ccc1F,[<rdkit.Chem.rdchem.Mol object at 0x29a203f90>...,JVLRHPWRWSSGDH-UHFFFAOYSA-N-SYTESp-base-A
6,batch1,MBMRUNGQBMGNOP-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0310A+A71EV2A-x0732A,Cc1cc(NC(=O)CO)cc(NC(=O)CC#N)c1,[<rdkit.Chem.rdchem.Mol object at 0x29a203f90>...,MBMRUNGQBMGNOP-UHFFFAOYSA-N-3ncoui-base-A
8,batch3,QDEXMOJNLWMDJW-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0528A,CN1CCC(Oc2ccccc2OCC(=O)N2CCOCC2)C1=O,[<rdkit.Chem.rdchem.Mol object at 0x299582660>...,QDEXMOJNLWMDJW-UHFFFAOYSA-N-3nJyVp-base-B
9,batch3,QDEXMOJNLWMDJW-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0528A,CN1CCC(Oc2ccccc2OCC(=O)N2CCOCC2)C1=O,[<rdkit.Chem.rdchem.Mol object at 0x299582660>...,QDEXMOJNLWMDJW-UHFFFAOYSA-N-3nJyVp-base-A
10,batch3,BRPRTCVLXGUWHL-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0528A,Cc1ccc(OC2CCN(C)C2=O)c(OCC(=O)N2CCOCC2)c1,[<rdkit.Chem.rdchem.Mol object at 0x299582660>...,BRPRTCVLXGUWHL-UHFFFAOYSA-N-3QVTyu-base-B
11,batch3,BRPRTCVLXGUWHL-UHFFFAOYSA-N,False,DistanceError: A71EV2A-x0528A,Cc1ccc(OC2CCN(C)C2=O)c(OCC(=O)N2CCOCC2)c1,[<rdkit.Chem.rdchem.Mol object at 0x299582660>...,BRPRTCVLXGUWHL-UHFFFAOYSA-N-3QVTyu-base-A


In [142]:
start_time = time.time()  # Start timing
# Get pdbblock from template_path
with open('/Users/kate_fieseler/PycharmProjects/EV-A71-2A-syndirella-run/fragments/x0310_relaxed_apo.pdb') as fh:
    pdbblock: str = fh.read()
lab = Laboratory(pdbblock=pdbblock, covalent_resi=None, run_plip=False)
placements: pd.DataFrame = lab.place(place_input_validator(df), n_cores=4, timeout=500)
end_time = time.time()  # End timing
elapsed_time = end_time - start_time  # Calculate elapsed time
print(f"Placing {len(df)} run time: {elapsed_time:.2f} seconds")  # Print the elapsed time

YMFXTZASBGMLKF-UHFFFAOYSA-N-touSdo-base-A - failed to minimise without constraints:  33.60827026910445 kcal/mol.


Placing 25 run time: 118.32 seconds


In [143]:
placements

,smiles,name,binary_hits,error,mode,∆∆G,∆G_bound,∆G_unbound,comRMSD,N_constrained_atoms,...,LE,unminimized_mol,minimized_mol,hit_mols,hit_names,percent_hybrid,largest_ring,N_HA,N_rotatable_bonds,outcome
0,N#CCC(=O)Nc1cccc(NC(=O)CO)c1,IGUPJTYXYZPOEC-UHFFFAOYSA-N-4Sratv-base-A,"[b""\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\...",,expansion,-8.119004,-7.585678,0.533325,0.778706,14,...,0.451056,<rdkit.Chem.rdchem.Mol object at 0x29aa43770>,<rdkit.Chem.rdchem.Mol object at 0x29aacb310>,[<rdkit.Chem.rdchem.Mol object at 0x29aae0ae0>...,"[A71EV2A-x0310A, A71EV2A-x0732A]",0,6,17,5,acceptable
1,Cc1ccc(NC(=O)CC#N)cc1NC(=O)CO,NBJGZNPZLLTJSQ-UHFFFAOYSA-N-4DiKxh-base-A,"[b""\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\...",,expansion,-4.189558,-3.566629,0.622929,0.892940,17,...,0.199503,<rdkit.Chem.rdchem.Mol object at 0x29a307180>,<rdkit.Chem.rdchem.Mol object at 0x29aae0ea0>,[<rdkit.Chem.rdchem.Mol object at 0x29aae04f0>...,"[A71EV2A-x0310A, A71EV2A-x0732A]",30,6,18,6,acceptable
2,N#CCC(=O)Nc1ccnc(NC(=O)CO)c1,ODONBPGOOMWOPQ-UHFFFAOYSA-N-nEzAW3-base-A,"[b""\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\...",,expansion,-4.815455,-4.173664,0.641790,0.811524,15,...,0.253445,<rdkit.Chem.rdchem.Mol object at 0x29aa43b30>,<rdkit.Chem.rdchem.Mol object at 0x29aae0f40>,[<rdkit.Chem.rdchem.Mol object at 0x29aae0d10>...,"[A71EV2A-x0310A, A71EV2A-x0732A]",15,6,17,5,acceptable
3,Cc1ccc(NC(=O)CO)cc1NC(=O)CC#N,TYJSTJNCXZDPLY-UHFFFAOYSA-N-CMcr7A-base-A,"[b""\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\...",,expansion,-9.009984,-8.462404,0.547580,0.828859,16,...,0.429047,<rdkit.Chem.rdchem.Mol object at 0x29aa439f0>,<rdkit.Chem.rdchem.Mol object at 0x29aae0720>,[<rdkit.Chem.rdchem.Mol object at 0x29aae09a0>...,"[A71EV2A-x0310A, A71EV2A-x0732A]",25,6,18,6,acceptable
4,N#CCC(=O)Nc1cc(NC(=O)CO)ccc1F,JVLRHPWRWSSGDH-UHFFFAOYSA-N-SYTESp-base-A,"[b""\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\...",,expansion,-5.373584,-4.677690,0.695894,0.834749,15,...,0.268679,<rdkit.Chem.rdchem.Mol object at 0x29aa4b860>,<rdkit.Chem.rdchem.Mol object at 0x29aae0c20>,[<rdkit.Chem.rdchem.Mol object at 0x29aae05e0>...,"[A71EV2A-x0310A, A71EV2A-x0732A]",17,6,18,5,acceptable
5,Cc1cc(NC(=O)CO)cc(NC(=O)CC#N)c1,MBMRUNGQBMGNOP-UHFFFAOYSA-N-3ncoui-base-A,"[b""\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\...",,expansion,-3.549765,-2.952109,0.597656,0.669248,14,...,0.186830,<rdkit.Chem.rdchem.Mol object at 0x29a9f8ae0>,<rdkit.Chem.rdchem.Mol object at 0x29aae09f0>,[<rdkit.Chem.rdchem.Mol object at 0x29aacef40>...,"[A71EV2A-x0310A, A71EV2A-x0732A]",0,6,18,6,acceptable
6,CN1CCC(Oc2ccccc2OCC(=O)N2CCOCC2)C1=O,QDEXMOJNLWMDJW-UHFFFAOYSA-N-3nJyVp-base-B,[b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\...,,expansion,-7.190722,-4.742991,2.447731,1.358683,27,...,0.266323,<rdkit.Chem.rdchem.Mol object at 0x29a9f84a0>,<rdkit.Chem.rdchem.Mol object at 0x29aae06d0>,[<rdkit.Chem.rdchem.Mol object at 0x29aae10e0>...,"[A71EV2A-x0528A, A71EV2A-x0739A]",43,6,24,6,too moved
7,CN1CCC(Oc2ccccc2OCC(=O)N2CCOCC2)C1=O,QDEXMOJNLWMDJW-UHFFFAOYSA-N-3nJyVp-base-A,[b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\...,,expansion,-4.033053,-2.982166,1.050888,1.460602,27,...,0.144038,<rdkit.Chem.rdchem.Mol object at 0x29a9f8450>,<rdkit.Chem.rdchem.Mol object at 0x29aae0180>,[<rdkit.Chem.rdchem.Mol object at 0x29aae1630>...,"[A71EV2A-x0528A, A71EV2A-x0739A]",43,6,24,6,too moved
8,Cc1ccc(OC2CCN(C)C2=O)c(OCC(=O)N2CCOCC2)c1,BRPRTCVLXGUWHL-UHFFFAOYSA-N-3QVTyu-base-B,[b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\...,,expansion,-4.666303,-3.140444,1.525859,1.532212,27,...,0.160907,<rdkit.Chem.rdchem.Mol object at 0x29a9f8b30>,<rdkit.Chem.rdchem.Mol object at 0x29aae0d60>,[<rdkit.Chem.rdchem.Mol object at 0x29aae1310>...,"[A71EV2A-x0528A, A71EV2A-x0739A]",43,6,25,7,too moved
9,Cc1ccc(OC2CCN(C)C2=O)c(OCC(=O)N2CCOCC2)c1,BRPRTCVLXGUWHL-UHFFFAOYSA-N-3QVTyu-base-A,[b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\...,,expansion,-5.405365,-3.935997,1.469368,1.330938,27,...,0.193049,<rdkit.Chem.rdchem.Mol object at 0x29aa6b7c0>,<rdkit.Chem.rdchem.Mol objec

In [144]:
# Save placements
placements.to_pickle('placements.pkl')
placements.to_csv('placements.csv')

In [1]:
import pkg_resources, sys, platform

get_version = lambda name: pkg_resources.get_distribution(name).version

print(f'Python {sys.version}\n'+
      f'on {platform.system()} {platform.machine()}\n'+
      f'with Fragmenstein {get_version("fragmenstein")} and PyRosetta  {get_version("pyrosetta")}'
     )

Python 3.9.18 (main, Sep 11 2023, 08:25:10) 
[Clang 14.0.6 ]
on Darwin arm64
with Fragmenstein 0.14.6 and PyRosetta  2023.21+release.9b3660a8932
